# PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
import ast

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from parsing import *
from outliers import *
from encodings import *

In [2]:
# df = pd.read_csv('madrid.csv')
# df.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
# df['price'] = df['price'].map(lambda x: tryInt(x))
# df['characteristics'] = df['characteristics'].apply(tryLiteralEval)

In [4]:
# data = freeChurro(df)

In [5]:
# df_characteristics = pd.DataFrame(data)

In [6]:
# df = pd.concat([df, df_characteristics], axis = 1).drop('characteristics', axis = 1)

In [7]:
df_ = pd.read_csv('madrid_preprocessed.csv')

In [8]:
df_sample = df_.sample(round(df_.shape[0]*0.15), random_state=42)
df_ = df_.reset_index()[~df_.reset_index()['index'].isin(df_sample.reset_index()['index'])].drop(['index'], axis=1)

In [9]:
# df_num = df_[['updated', 'surface']]
# df_num = dropOutliers(df_num, 1.5)
# df_num

In [10]:
# df_ = pd.concat([df_num, df_.drop(['updated', 'surface'], axis = 1)], axis = 1)
# df_ = df_[~df_['updated'].isna()]
# df_ = df_[~df_['surface'].isna()]

In [11]:
df_ = df_[df_['price'].between(50_000, 5_000_000)]
df_ = df_[df_['bathrooms'] <= 15]
df_ = df_[df_['surface'] <= 1_000]
df_

,price,lat,lng,updated,type,bulevar,calle,avenida,parque,autovia,...,publisher,age,garage,lift,surface,net_surface,garden,rooms,condition,bathrooms
1,116500.0,40.402079,-3.702151,1.674947e+09,Apartamento,2,2,2,2,2,...,inmobiliaria,50.0,no,yes,25.0,NaN,no,NaN,NaN,1.0
2,169000.0,40.534457,-3.479415,1.673392e+09,Apartamento,0,0,0,0,0,...,inmobiliaria,NaN,no,yes,70.0,67.0,no,1.0,En buen estado,1.0
4,130000.0,40.347096,-3.827826,1.672615e+09,Apartamento,0,0,0,0,0,...,inmobiliaria,NaN,no,yes,60.0,54.0,no,2.0,NaN,1.0
5,88000.0,40.031830,-3.599734,1.674947e+09,Apartamento,2,2,2,2,2,...,inmobiliaria,10.0,no,yes,37.0,30.0,no,1.0,En buen estado,1.0
6,115000.0,40.031169,-3.598471,1.672874e+09,Apartamento,2,2,2,2,2,...,inmobiliaria,10.0,no,yes,40.0,40.0,no,1.0,En buen estado,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15837,189450.0,40.441125,-3.466229,1.672960e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,no,no,88.0,NaN,no,2.0,A estrenar,2.0
15840,156800.0,40.453200,-3.472508,1.672787e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,no,yes,90.0,NaN,no,3.0,NaN,1.0
15841,135000.0,40.454606,-3.455234,1.672960e+09,Piso,1,1,1,1,1,...,inmobiliaria,NaN,no,yes,67.0,62.0,no,3.0,A reformar,1.0
15842,210000.0,40.441928,-3.473036,1.672615e+09,Piso,1,1,1,1,1,...,inmobiliaria,NaN,yes,yes,79.0,63.0,no,1.0,En buen estado,2.0


In [12]:
# df_text = df[['Cocina', 'Cocina equipada', 'Superficie solar', 'Sistema de seguridad', 'Portero automático',
#                       'Aire acondicionado', 'Puerta blindada', 'Amueblado', 'Trastero', 'Calefacción', 'Gas',
#                       'Tipo de casa', 'Piscina', 'Vidrios dobles', 'Planta', 'Tipo suelo', 'Balcón', 'Orientación',
#                       'Armarios empotrados', 'Comedor']]

# df_text['concatenated'] = df_text.apply(lambda row: ' '.join([str(val) if pd.notna(val) else '' for val in row]), axis=1)
# df_text['concatenated'] = df_text['concatenated'].apply(lambda x: x.strip())
# df_text['concatenated'] = df_text['concatenated'].apply(lambda x: ' '.join([y.strip() for y in x.split()]))

# texts = df_text['concatenated'].to_list()

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(texts)

In [13]:
# df_text[['concatenated']]

In [14]:
# def get_best_k(X, n_k, model = KMeans()):
#     inertias = list()
    
#     for k in n_k:
#         model.n_clusters = k
#         model.fit(X)
#         inertias.append(model.inertia_)
        
#     k_i = np.array([[k, i] for k, i in enumerate(inertias, start = 1)])

#     k_i_max = np.max(k_i, axis=0)
#     k_i_norm = k_i / k_i_max
    
#     distances = [np.linalg.norm([point, [0,0]]) for point in k_i_norm]
    
#     return pd.concat([pd.DataFrame([x for x in range(1,n_k[-1] + 1)], columns = ['K']), pd.DataFrame(distances, columns = ['Distance'])], axis = 1).sort_values('Distance')

In [15]:
# best_k = get_best_k(X,range(1,61)).reset_index(drop = True)['K'].iloc[0]

In [16]:
# kmeans = KMeans(n_clusters = best_k)
# kmeans.fit(X)

In [17]:
# df_['cluster'] = kmeans.labels_
# df_ = pd.concat([df_, pd.DataFrame(kmeans.labels_)], axis = 1)
# df_ = df_[~df_['price'].isna()]
# df_ = df_.rename(columns = {0 : 'cluster'})
# df_

In [18]:
# df_.info()

In [19]:
df = df_.copy()

# type encoding

In [20]:
# types = df['type'].unique()
# types_dict = {type_ : x for type_, x in zip(types, range(len(types)))}

In [21]:
# types_dict = {'piso' : 1}

In [22]:
# df['type'] = df['type'].replace(types_dict)

In [23]:
# df['type'] = df['type'].apply(lambda x : 0 if x != 'piso' else x)

In [24]:
# df_type, encodings_type = targetEncoding(df[['type']], df['price'], condition = 'target_mean')
df_type, encodings_type = frequencyEncoding(df[['type']])
df['type'] = df_type['type']

NameError: name 'frequencyEncoding' is not defined

# garage, lift, garden and publisher encoding

In [ ]:
df['garage'] = df['garage'].apply(lambda x: 1 if x == 'yes' else 0)
df['lift'] = df['lift'].apply(lambda x: 1 if x == 'yes' else 0)
df['garden'] = df['garden'].apply(lambda x: 1 if x == 'yes' else 0)
df['publisher'] = df['publisher'].apply(lambda x: 1 if x == 'inmobiliaria' else 0)

# condition encoding

In [ ]:
conditions = df['condition'].unique()
conditions_dict = {cond_ : x for cond_, x in zip(conditions, range(len(conditions)))}

In [ ]:
df['condition'] = df['condition'].replace(conditions_dict)

In [ ]:
df_surface = df[['surface', 'net_surface']].dropna()
np.corrcoef(df_surface['surface'], df_surface['net_surface'])

array([[1.        , 0.97416502],
       [0.97416502, 1.        ]])

In [ ]:
df

,price,lat,lng,updated,type,bulevar,calle,avenida,parque,autovia,...,publisher,age,garage,lift,surface,net_surface,garden,rooms,condition,bathrooms
1,116500.0,40.402079,-3.702151,1.674947e+09,655023.491228,2,2,2,2,2,...,1,50.0,0,1,25.0,NaN,0,NaN,0,1.0
2,169000.0,40.534457,-3.479415,1.673392e+09,655023.491228,0,0,0,0,0,...,1,NaN,0,1,70.0,67.0,0,1.0,1,1.0
4,130000.0,40.347096,-3.827826,1.672615e+09,655023.491228,0,0,0,0,0,...,1,NaN,0,1,60.0,54.0,0,2.0,0,1.0
5,88000.0,40.031830,-3.599734,1.674947e+09,655023.491228,2,2,2,2,2,...,1,10.0,0,1,37.0,30.0,0,1.0,1,1.0
6,115000.0,40.031169,-3.598471,1.672874e+09,655023.491228,2,2,2,2,2,...,1,10.0,0,1,40.0,40.0,0,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15837,189450.0,40.441125,-3.466229,1.672960e+09,451210.714974,0,0,0,0,0,...,1,NaN,0,0,88.0,NaN,0,2.0,3,2.0
15840,156800.0,40.453200,-3.472508,1.672787e+09,451210.714974,0,0,0,0,0,...,1,NaN,0,1,90.0,NaN,0,3.0,0,1.0
15841,135000.0,40.454606,-3.455234,1.672960e+09,451210.714974,1,1,1,1,1,...,1,NaN,0,1,67.0,62.0,0,3.0,2,1.0
15842,210000.0,40.441928,-3.473036,1.672615e+09,451210.714974,1,1,1,1,1,...,1,NaN,1,1,79.0,63.0,0,1.0,1,2.0


# TRAINING

In [ ]:
df = df[~df['price'].isna()]

X = df.drop(['price', 'net_surface'], axis = 1)
y = df[['price']]

imputer = KNNImputer(n_neighbors=3)

X_imputed = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size = 0.2, random_state = 42)

model = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100)

model.fit(X_train, y_train)

yhat = model.predict(X_test)

print(r2_score(y_test, yhat))
print(mean_absolute_error(y_test, yhat))
print(mean_squared_error(y_test, yhat))

C:\Users\Aa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8414067968193032
112662.80783427358
60280034313.09107


In [ ]:
df.drop('price', axis = 1).columns

Index(['lat', 'lng', 'updated', 'type', 'bulevar', 'calle', 'avenida',
       'parque', 'autovia', 'plaza', 'paseo', 'carretera', 'publisher', 'age',
       'garage', 'lift', 'surface', 'net_surface', 'garden', 'rooms',
       'condition', 'bathrooms'],
      dtype='object')

In [ ]:
# feature_importances = {y : x for x, y in zip(model.feature_importances_, X.columns)}

In [ ]:
# def extractImportances(data, percentage):
#     # Sort the dictionary items by values in descending order
#     sorted_items = sorted(data.items(), key=lambda x: x[1], reverse=True)

#     cumulative_sum = 0
#     selected_keys = []

#     for key, value in sorted_items:
#         print(key, value)
#         cumulative_sum += value
#         selected_keys.append(key)

#         if cumulative_sum >= percentage:
#             break

#     return selected_keys

In [ ]:
# importances = extractImportances(feature_importances, 0.9)

In [ ]:
# X = df.drop(['price'], axis = 1)[importances]
# y = df[['price']]

# imputer = KNNImputer(n_neighbors=3)

# X_imputed = imputer.fit_transform(X)

# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size = 0.2, random_state = 42)

# model = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100)

# model.fit(X_train, y_train)

# yhat = model.predict(X_test)

# print(r2_score(y_test, yhat))
# print(mean_absolute_error(y_test, yhat))
# print(mean_squared_error(y_test, yhat))

In [ ]:
# importances

In [ ]:
# importances = extractImportances(feature_importances, 0.8)

In [ ]:
# X = df.drop(['price'], axis = 1)[importances]
# y = df[['price']]

# imputer = KNNImputer(n_neighbors=3)

# X_imputed = imputer.fit_transform(X)

# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size = 0.2, random_state = 42)

# model = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100)

# model.fit(X_train, y_train)

# yhat = model.predict(X_test)

# print(r2_score(y_test, yhat))
# print(mean_absolute_error(y_test, yhat))
# print(mean_squared_error(y_test, yhat))

In [ ]:
# importances

# Algoritmos

In [ ]:
# models = {'LR' : LinearRegression(n_jobs = -1),
#           'DT' : DecisionTreeRegressor(random_state = 42),
#           'RF' : RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100),
#           'SVR' : SVR(),
#           'KN' : KNeighborsRegressor(n_jobs = -1, n_neighbors = 3),}

# metrics_cols = ['model', 'r2_score', 'mean_squared_error']
# metrics = pd.DataFrame(columns = metrics_cols)

# df = df[~df['price'].isna()]

# X = df.drop(['price', 'net_surface'], axis = 1)
# y = df[['price']]

# imputer = KNNImputer(n_neighbors=3)
# X_imputed = imputer.fit_transform(X)

# for enum, m in enumerate(models.keys()):
    
#     model = models[m]
    
#     if m == 'SVR' or m == 'KN':
#       X_scaler = MinMaxScaler()
#       X_imputed = X_scaler.fit_transform(X_imputed)
#       y_scaler = MinMaxScaler()
#       y = y_scaler.fit_transform(y)
      
#     X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size = 0.2, random_state = 42)
      
#     model.fit(X_train, y_train)
#     yhat = model.predict(X_test)
    
#     updating_metrics = pd.DataFrame([[model, r2_score(y_test, yhat), mean_squared_error(y_test, yhat)]], columns = metrics_cols)
#     metrics = pd.concat([metrics, updating_metrics], axis = 0, ignore_index = True)
    
#     print(f'({enum+1}) - {models[m]} completed')
#     print('---'*30)

In [ ]:
# metrics.sort_values('r2_score', ascending = False)

In [ ]:
df_sample

,price,lat,lng,updated,type,bulevar,calle,avenida,parque,autovia,...,publisher,age,garage,lift,surface,net_surface,garden,rooms,condition,bathrooms
8466,226000.0,40.729647,-3.582455,1.672615e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,yes,yes,150.0,108.0,no,3.0,NaN,2.0
2764,428500.0,40.442969,-3.464236,1.673997e+09,Casa,1,1,1,1,1,...,inmobiliaria,NaN,no,no,177.0,NaN,no,4.0,A estrenar,3.0
10979,479000.0,40.510236,-3.694039,1.674861e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,yes,yes,120.0,84.0,no,3.0,En buen estado,2.0
2183,1350000.0,40.413838,-3.791630,1.672787e+09,Casa,0,0,0,0,0,...,inmobiliaria,NaN,yes,no,450.0,NaN,no,6.0,En buen estado,5.0
11882,210190.0,40.229607,-3.749955,1.675120e+09,Piso,1,1,1,1,1,...,inmobiliaria,NaN,yes,yes,72.0,NaN,no,2.0,A estrenar,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,99950.0,40.296712,-3.302587,1.672614e+09,Casa,0,0,0,0,0,...,inmobiliaria,NaN,no,no,248.0,NaN,no,5.0,En buen estado,3.0
4121,350000.0,40.742860,-3.511755,1.672960e+09,Chalet,0,0,0,0,0,...,inmobiliaria,30.0,yes,no,182.0,149.0,yes,4.0,En buen estado,2.0
9365,480000.0,40.674201,-4.087623,1.674256e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,no,yes,340.0,NaN,no,5.0,NaN,3.0
5955,145000.0,40.448851,-3.705134,1.672874e+09,Piso,0,0,0,0,0,...,inmobiliaria,NaN,no,no,40.0,NaN,no,NaN,En buen estado,1.0


In [ ]:
df_sample['type'] = df_sample['type'].replace(encodings_type['type'])

In [ ]:
# df_sample['type'] = df_sample['type'].apply(lambda x : 0 if x != 'piso' else x)

In [ ]:
df_sample['garage'] = df_sample['garage'].apply(lambda x: 1 if x == 'yes' else 0)
df_sample['lift'] = df_sample['lift'].apply(lambda x: 1 if x == 'yes' else 0)
df_sample['garden'] = df_sample['garden'].apply(lambda x: 1 if x == 'yes' else 0)
df_sample['publisher'] = df_sample['publisher'].apply(lambda x: 1 if x == 'inmobiliaria' else 0)

In [ ]:
df_sample['condition'] = df_sample['condition'].replace(conditions_dict)

In [ ]:
df_surface = df_sample[['surface', 'net_surface']].dropna()
np.corrcoef(df_surface['surface'], df_surface['net_surface'])

array([[1.        , 0.63535391],
       [0.63535391, 1.        ]])

In [ ]:
df_validation = df_sample.dropna()

In [ ]:
# X = df_validation[importances]
X = df_validation.drop(['price', 'net_surface'], axis = 1)
y = df_validation['price']

yhat = model.predict(X)

print(r2_score(y, yhat))
print(mean_absolute_error(y, yhat))
print(mean_squared_error(y, yhat))

0.8025561754032962
94996.84013097778
63646369029.51799


C:\Users\Aa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
